## Imports

In [7]:
import numpy as np
import pandas as pd
from ape import accounts, chain, Contract, networks, project

In [8]:
networks.parse_network_choice('ethereum:mainnet:alchemy').__enter__()

<alchemy chain_id=1>

## Contract objects

In [9]:
# Load Uniswap factory contract
factory = Contract("0x1F98431c8aD98523631AE4a59f267346ea31F984")

# Load a referece pool; obtain `contract_type` from here to read any uniswap pool with ape
pool_ref = Contract('0x8f8EF111B67C04Eb1641f5ff19EE54Cda062f163')

# USDC/ETH 0.05%
pool = Contract("0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640", contract_type=pool_ref.contract_type)

token0 = Contract(pool.token0())
token1 = Contract(pool.token1())

## Get prices

#### Inputs for querying and saving price data

In [10]:
start_block = 12376729  # ETH/USDC 0.05% pool deployment block
# start_block = chain.blocks.head.number - 100_000
head_block = chain.blocks.head.number
step = 2400
filename = f"data/price_{pool.address}_{start_block}_{head_block}_{step}.csv"

#### Helpful functions

In [11]:
# calculate prices for both y/x and x/y in case of "weird" token0, token1 ordering
def price(sqrt_price_x96) -> int:
    return (int(sqrt_price_x96) >> 96) ** 2

#### Query prices

In [12]:
# query for tick data each block and insert into pandas dataframe
is_head = True
y_decimals = token1.decimals()
for block in range(start_block, head_block, step):
    print(f"Processing block {block} ...", end='\r')
    
    # get the sqrt price data at block
    slot0 = pool.slot0(block_identifier=block)
    y_by_x = price(slot0.sqrtPriceX96)
    row = {
        'block_number': [block],
        'timestamp': [chain.blocks[block].timestamp],
        'sqrt_price_x96': [slot0.sqrtPriceX96],
        'y/x': [y_by_x],
        'x/y': 10 ** y_decimals // y_by_x
    }
        
    # convert to pd dataframe then append to file
    df = pd.DataFrame(data=row)
    df.to_csv(filename, mode='a', index=False, header=is_head)
        
    if is_head:
        is_head = False

In [13]:
df = pd.read_csv(filename)
df.head()

,block_number,timestamp,sqrt_price_x96,y/x,x/y
0,12376729,1620250931,1350174849792634181862360983626536,290395681,3443577385
1,12379129,1620282250,1349455942403455280869650838376902,290089024,3447217637
2,12381529,1620314849,1331645075167582260580421920153591,282475249,3540133174
3,12383929,1620346823,1337727823580104074746991547077862,285069456,3507917031
4,12386329,1620379299,1352858712162969376165151222026880,291555625,3429877231


In [14]:
df.shape

(2338, 5)